# Homework 1

## Dependencies

In [ ]:
!pip install ipympl

In [ ]:
import csv
from matplotlib import pyplot as plt
import seaborn as sns
import typing as t
import numpy as np
import numpy.typing as npt
import itertools
import pandas as pd
from scipy import stats as scpy_stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

%matplotlib widget
%matplotlib inline

In [ ]:
class Person:
    Gender: str
    Age: int
    Height: float
    Weight: float
    family_history_with_overweight: str
    FAVC: str
    FCVC: int
    NCP: int
    CAEC: str
    SMOKE: str
    CH2O: int
    SCC: str
    FAF: str
    TUE: int
    CALC: str
    MTRANS: str
    NObeyesdad: str

    def __init__(
        self,
        Gender: str,
        Age: int,
        Height: float,
        Weight: float,
        family_history_with_overweight: str,
        FAVC: str,
        FCVC: int,
        NCP: int,
        CAEC: str,
        SMOKE: str,
        CH2O: int,
        SCC: str,
        FAF: int,
        TUE: int,
        CALC: str,
        MTRANS: str,
        NObeyesdad: str,
    ):
        self.Gender = Gender
        self.Age = Age
        self.Height = Height
        self.Weight = Weight
        self.family_history_with_overweight = family_history_with_overweight
        self.FAVC = FAVC
        self.FCVC = FCVC
        self.NCP = NCP
        self.CAEC = CAEC
        self.SMOKE = SMOKE
        self.CH2O = CH2O
        self.SCC = SCC
        self.FAF = FAF
        self.TUE = TUE
        self.CALC = CALC
        self.MTRANS = MTRANS
        self.NObeyesdad = NObeyesdad

    def __str__(self):
        return (
            "{"
            + f'"Gender": "{self.Gender}",'
            + f'"Age": {self.Age},'
            + f'"Height": {self.Height},'
            + f'"Weight": {self.Weight},'
            + f'"family_history_with_overweight": "{self.family_history_with_overweight}",'
            + f'"FAVC": "{self.FAVC}",'
            + f'"FCVC": {self.FCVC},'
            + f'"NCP": {self.NCP},'
            + f'"CAEC": "{self.CAEC}",'
            + f'"SMOKE": "{self.SMOKE}",'
            + f'"CH2O": {self.CH2O},'
            + f'"SCC": {self.SCC},'
            + f'"FAF": "{self.FAF}",'
            + f'"TUE": {self.TUE},'
            + f'"CALC": "{self.CALC}",'
            + f'"MTRANS": "{self.MTRANS}",'
            + f'"NObeyesdad": "{self.NObeyesdad}"'
            + "}"
        )

    def __len__(self):
        return 17

    __repr__ = __str__

In [ ]:
NUMERICAL_VARIABLES = ["Age", "Height", "Weight", "FCVC", "NCP", "CH2O", "FAF", "TUE"]
CATEGORICAL_VARIABLES = ["FAVC", "CAEC", "CALC", "SCC", "MTRANS", "Gender", "family_history_with_overweight", "SMOKE", "NObeyesdad"]

CATEGORICAL_VARIABLES_NO_LABEL = ["FAVC", "CAEC", "CALC", "SCC", "MTRANS", "Gender", "family_history_with_overweight", "SMOKE"]

LABEL_VARIABLE = "NObeyesdad"

In [ ]:
class DatasetManager:
    def __init__(self, path_to_csv: str):
        self.path_to_csv = path_to_csv

    def load_as_obj_list(self) -> list[Person]:
        with open(self.path_to_csv) as csv_file:
            csv_reader = csv.DictReader(csv_file)
            return [Person(**row) for row in csv_reader]

    @staticmethod

    @staticmethod
    def obj_list_to_flat_np_array(
        data: list[Person],
        attrs_list: list[str] = numeric_variables + categorical_values,
    ) -> npt.NDArray[np.float32]:
        return np.array(
            [
                np.float32(getattr(entry, field))
                for entry in data
                for field in attrs_list
            ]
        )

    @staticmethod
    def obj_list_to_list(
        data: t.List[Person], attrs_list: t.List[str] = categorical_values
    ) -> t.List[t.List[str]]:
        return [[getattr(entry, field) for field in attrs_list] for entry in data]

    @staticmethod
    def obj_list_to_flat_list(
        data: t.List[Person], attrs_list: t.List[str] = categorical_values
    ) -> t.List[str]:
        return [getattr(entry, field) for field in attrs_list for entry in data]

    @staticmethod
    def make_combinations(
        attrs_list: t.List[str], k: int = 2
    ) -> t.List[t.Tuple[str, ...]]:
        return [subset for subset in itertools.combinations(attrs_list, k)]

    def obj_list_to_np_array(data: list[Person], attrs_list: list[str] = NUMERICAL_VARIABLES + CATEGORICAL_VARIABLES) -> np.array:
        return np.array([[getattr(entry, field) for field in attrs_list] for entry in data])

    @staticmethod
    def obj_list_to_np_array_numeric(data: list[Person], attrs_list: list[str] = NUMERICAL_VARIABLES + CATEGORICAL_VARIABLES) -> np.array:
        return DatasetManager.obj_list_to_np_array(data, attrs_list).astype(np.float64)

    @staticmethod
    def obj_list_to_np_array_category(data: list[Person], attrs_list: list[str] = NUMERICAL_VARIABLES + CATEGORICAL_VARIABLES) -> np.array:
        data_pd = pd.DataFrame(DatasetManager.obj_list_to_np_array(data, attrs_list))
        return np.vstack([pd.factorize(data_pd[col])[0] for col in data_pd.columns]).T

    @staticmethod
    def obj_list_to_np_array_category_binary(data: list[Person], attrs_list: list[str]) -> np.array:
        data_pd = pd.DataFrame(DatasetManager.obj_list_to_np_array(data, attrs_list))
        return pd.get_dummies(data_pd).to_numpy().astype(np.float64)

    @staticmethod
    def object_list_to_pd_dataframe_category(data: list[Person], attr: str) -> pd.Series:
        return pd.DataFrame(DatasetManager.obj_list_to_np_array(data, [attr])).value_counts()

    @staticmethod
    def object_list_to_pd_dataframe_contingency_table(data: list[Person], lh_attr: str, rh_attr: str) -> pd.DataFrame:
        data_pd = pd.DataFrame(DatasetManager.obj_list_to_np_array(data, [lh_attr, rh_attr]), columns=[lh_attr, rh_attr])
        return pd.crosstab(data_pd[lh_attr], data_pd[rh_attr], margins=False)

    @staticmethod
    def get_numerical_for_each_category(data: list[Person], numerical_attr: str, categorical_attr: str) -> list[tuple[np.array, str]]:

        data_np = DatasetManager.obj_list_to_np_array(data, [numerical_attr, categorical_attr])
        unique_categorical_values = np.unique(data_np[:, 1]).tolist()

        return [(data_np[data_np[:, 1] == unique_val, 0].astype(np.float64), unique_val) for unique_val in unique_categorical_values]


In [ ]:
dataset_manager = DatasetManager("data/ObesityDataSet.csv")
dataset_obj_list = dataset_manager.load_as_obj_list()

## Univariate analysis

The following will be applied:
1. central tendency
1. spread
1. distribution form (skewness, kurtosis)
1. frequency of categorical data
1. graphs
    1. histograms
    1. density
    1. boxplots

### 1. Central tendency

Calculates mean, median and mode for each data series.

In [ ]:
def calculate_central_tendency_numerical(
    np_dataset: np.array,
) -> t.Tuple[float, float, float]:
    mean = np.mean(np_dataset)
    median = np.median(np_dataset)
    mode = scpy_stats.mode(np_dataset).mode

    return mean, median, mode

In [ ]:
for numerical_var in NUMERICAL_VARIABLES:
    dataset_for_numerical_val = DatasetManager.obj_list_to_flat_np_array(
        dataset_obj_list, [numerical_var]
    )

    mean, median, mode = calculate_central_tendency_numerical(dataset_for_numerical_val)

    print(f"On numerical var {numerical_var}")
    print(f"Mean: {mean}")
    print(f"Median: {median}")
    print(f"Mode: {mode}\n{['-' * 10]}")
    # MODE nu cred ca e si la variabile numerice

### 2. Spread

Calculates the spread of data for each data series. Useful to know wether the data has a "central tendency".

In [ ]:
def calculate_spread_numerical(np_dataset: np.array) -> t.Tuple[float, float, float]:
    dataset_range = np.ptp(np_dataset)
    dataset_variance = np.var(np_dataset)
    dataset_standard_deviation = np.std(np_dataset)

    return dataset_range, dataset_variance, dataset_standard_deviation

In [ ]:
for numerical_var in numeric_variables:
    dataset_for_numerical_val = DatasetManager.obj_list_to_flat_np_array(
        dataset_obj_list, [numerical_var]
    )
    dataset_range, dataset_variance, dataset_standard_deviation = (
        calculate_spread_numerical(dataset_for_numerical_val)
    )

    print(f"On numerical var {numerical_var}")
    print(f"Range: {dataset_range}")
    print(f"Variance: {dataset_variance}")
    print(f"Standard deviation: {dataset_standard_deviation}")
    print(f"{['-' * 10]}")

### 3. Skewness, kurtosis

Calculates skewness and kurtosis of the dataset.

Meaning for skewness:
* Positively skewed (right-skewed):
    * The distribution is positively skewed **if the distribution's tail on the right side is longer or "fatter" than the left side**. This means that there are more data points on the left side, and the distribution as a longer right tail.
    * Values: `> 1`
* Negatively skewed (left-skewed):
    * The distribution is negatively skewed **if the distribution's tail on the left side is longer or "fatter" than the right side**. This means that there are more data points on the right side, and the distribution as a longer left tail.
    * Values: `< -1`
* Symmetric: 
    * If the distribution is _roughly_ the same on both sides, it is symmetric, and the skewness is close to 0.
    * Values: `~ 0`

Meaning of kurtosis:
* Mesokurtic (Normal distribution):
    * A distribution with kurtosis similar to that of a normal distribution
    * Values: `~ 0`
* Leptokurtic:
    * A distribution with pisitive kurtosis, indicating heavier tails and a more peaked central region compared to a normal distribution
    * Values: `> 1`
* Platykurtic:
    * A normal distribution with a negative kurtosis, indicating lighter tails and a flatter central region compared to a normal distribution
    * Values: `< -1`

In [ ]:
def calculate_skewness_kurtosis_numerical(
    np_dataset: np.array,
) -> t.Tuple[float, float]:
    dataset_skewness = scpy_stats.skew(np_dataset)
    dataset_kurtosis = scpy_stats.kurtosis(np_dataset)

    return dataset_skewness, dataset_kurtosis

In [ ]:
for numerical_var in numeric_variables:
    dataset_for_numerical_val = DatasetManager.obj_list_to_flat_np_array(
        dataset_obj_list, [numerical_var]
    )
    dataset_skewness, dataset_kurtosis = calculate_skewness_kurtosis_numerical(
        dataset_for_numerical_val
    )

    print(f"On numerical var {numerical_var}")
    print(f"Skewness: {dataset_skewness}")
    print(f"Kurtosis: {dataset_kurtosis}")
    print(f"{['-' * 10]}\n")

### 4. Frequency of categorical data

Here we count how often we see the categorical data in a data series.

In [ ]:
def calculate_frequency_of_data_categorical(dataset: t.List[str]) -> t.Dict[str, int]:
    counts = {}

    for entry in dataset:
        if entry in counts:
            counts[entry] += 1
        else:
            counts[entry] = 1

    return counts

In [ ]:
for categorical_var in categorical_values:
    dataset_for_categorical_val = DatasetManager.obj_list_to_flat_list(
        dataset_obj_list, [categorical_var]
    )
    dataset_frequency = calculate_frequency_of_data_categorical(
        dataset_for_categorical_val
    )

    print(f"On categorical var {categorical_var}")

    for entry in dataset_frequency:
        print(f'Frequency of value "{entry}": {dataset_frequency[entry]}')

    print(f"{['-' * 10]}")

### 5. Graphs

Here we can find histograms, density charts and boxplots.

#### 5.1. Histograms

Histograms plot how frequently we meet a data entry from the dataset.

In [ ]:
for numerical_var in numeric_variables:
    dataset_for_numerical_val = DatasetManager.obj_list_to_flat_np_array(
        dataset_obj_list, [numerical_var]
    )

    plt.figure()
    plt.hist(
        dataset_for_numerical_val,
        bins=30,
        color="lightblue",
        edgecolor="black",
        alpha=0.7,
    )
    plt.title(f'Histogram for numerical variable "{numerical_var}"')
    plt.show()

In [ ]:
for categorical_var in categorical_values:
    dataset_for_categorical_val = DatasetManager.obj_list_to_flat_list(
        dataset_obj_list, [categorical_var]
    )

    dataset_frequency = calculate_frequency_of_data_categorical(
        dataset_for_categorical_val
    )
    dataset_keys = [key for key in dataset_frequency]
    dataset_values = [dataset_frequency[key] for key in dataset_frequency]

    plt.figure()
    plt.bar(
        dataset_keys, dataset_values, color="lightblue", edgecolor="black", alpha=0.7
    )
    plt.title(f'Histogram for categorical variable "{categorical_var}"')
    plt.show()

#### 5.2. Density charts

Density charts plot how frequently we meet a data entry from the dataset and what distribution they follow.

In [ ]:
for numerical_var in numeric_variables:
    dataset_for_numerical_val = DatasetManager.obj_list_to_flat_np_array(
        dataset_obj_list, [numerical_var]
    )

    plt.figure()
    sns.kdeplot(dataset_for_numerical_val, bw=0.1)
    plt.hist(
        dataset_for_numerical_val,
        bins=30,
        density=True,
        color="lightblue",
        edgecolor="black",
        alpha=0.4,
    )

    plt.xlabel("Values")
    plt.ylabel("Density")
    plt.title(f'Density Chart for "{numerical_var}"')
    plt.show()

#### 5.3. Boxplots

Boxplots charts show how the data "behaves":
* min
* max
* quantiles
* outliers
* median
* inter-quartile range (contains 50% of the data)
* Skewness of data
* Robustness to extreme values
* etc.

In [ ]:
for numerical_var in numeric_variables:
    dataset_for_numerical_val = DatasetManager.obj_list_to_flat_np_array(
        dataset_obj_list, [numerical_var]
    )

    plt.figure()
    plt.boxplot(dataset_for_numerical_val, labels=[numerical_var])
    plt.xlabel("Group")
    plt.ylabel("Values")
    plt.title(f'Boxplot for "{numerical_var}"')
    plt.show()

## Multivariate analysis

The following will be applied:
1. Find Pearson Correlation and Spearman (Rank) correlation
1. Phi squared test, Fisher test and contingency tables
1. T Test (Testul Mediilor), Z Test, ANOVA Test
1. PCA in 2D si 3D; TSNE in 2d si 3D; Both using numerical + categorial
1. Corrgrams

### 1. Find Pearson Correlation and Spearman (Rank) correlation

In [ ]:
pearson_correlation: list[tuple[str, str, float]] = []
rank_correlation: list[tuple[str, str, float]] = []

for it in range(len(NUMERICAL_VARIABLES) - 1):
    for jt in range(it + 1, len(NUMERICAL_VARIABLES)):
        lh_var, rh_var = NUMERICAL_VARIABLES[it], NUMERICAL_VARIABLES[jt]

        lh_dataset = DatasetManager.obj_list_to_np_array_numeric(dataset_obj_list, [lh_var]).reshape(-1)

        rh_dataset = DatasetManager.obj_list_to_np_array_numeric(dataset_obj_list, [rh_var]).reshape(-1)

        pearson_cr = scpy_stats.pearsonr(lh_dataset, rh_dataset).correlation
        rank_cr = scpy_stats.spearmanr(lh_dataset, rh_dataset).correlation

        pearson_correlation.append((lh_var, rh_var, pearson_cr))
        rank_correlation.append((lh_var, rh_var, rank_cr))

In [ ]:
pearson_correlation = list(sorted(pearson_correlation, key=lambda x: x[2]))
print(f"Pearson Correlation\n\n")
for item in pearson_correlation:
    print(f"{item}\n")

In [ ]:
rank_correlation = list(sorted(rank_correlation, key=lambda x: x[2]))
print(f"Rank Correlation\n\n")
for item in rank_correlation:
    print(f"{item}\n")

In [ ]:
best_numerical_correlations = pearson_correlation[:3] + pearson_correlation[-6:]
print(best_numerical_correlations)

### 2. Phi squared test, Fisher test and contingency tables

In [ ]:
alfa = 0.05

chi2_test: list[tuple[str, str, float]] = []
fisher_test: list[tuple[str, str, float]] = []

for it in range(len(CATEGORICAL_VARIABLES) - 1):
    for jt in range(it + 1, len(CATEGORICAL_VARIABLES)):
        lh_var, rh_var = CATEGORICAL_VARIABLES[it], CATEGORICAL_VARIABLES[jt]

        dataset = DatasetManager.object_list_to_pd_dataframe_contingency_table(dataset_obj_list, lh_var, rh_var).to_numpy()

        stat, p, degrees_of_freedom, expected_contingency = scpy_stats.chi2_contingency(dataset)

        if p <= alfa:
            print(f"{(lh_var, rh_var)} DEPENDENTE CHI2={stat}")
            chi2_test.append((lh_var, rh_var, stat))
        else:
            print(f"{(lh_var, rh_var)} INDEPENDENTE CHI2={stat}")

        if dataset.shape == (2, 2):
            odds, p_value = scpy_stats.fisher_exact(dataset)
            if p_value <= alfa:
                fisher_test.append((lh_var, rh_var, odds))


In [ ]:
chi2_test = list(sorted(chi2_test, key=lambda x: x[2]))
print(f"CHI2 Test\n\n")
for item in chi2_test:
    print(f"{item}\n")

In [ ]:
best_chi2_test = chi2_test[:-10]

In [ ]:
for lh_var, rh_var, chi2_val in best_chi2_test[::-1]:
    print(DatasetManager.object_list_to_pd_dataframe_contingency_table(dataset_obj_list, lh_var, rh_var))
    print("\n" + "".join(["-"] * 10) + "\n")

In [ ]:
fisher_test = list(sorted(fisher_test, key=lambda x: x[2]))
print(f"Fisher Test\n\n")
for item in fisher_test:
    print(f"{item}\n")

### 3. T Test (Testul Mediilor), Z Test, ANOVA Test

In [ ]:
alfa = 0.05

t_test: list[tuple[str, str, float]] = []
anova_test: list[tuple[str, str, float]] = []

for it in range(len(NUMERICAL_VARIABLES)):
    for jt in range(len(CATEGORICAL_VARIABLES)):
        lh_var, rh_var = NUMERICAL_VARIABLES[it], CATEGORICAL_VARIABLES[jt]

        numerical_for_each_category = DatasetManager.get_numerical_for_each_category(dataset_obj_list, lh_var, rh_var)

        if len(numerical_for_each_category) == 2:
            lh_numeric_data, rh_numeric_data = numerical_for_each_category[0][0], numerical_for_each_category[1][0]

            statistic, p_value = scpy_stats.ttest_ind(lh_numeric_data, rh_numeric_data)

            if p_value <= alfa:
                print(f"Found {(lh_var, rh_var)} TO BE correlated with T Test={statistic}")
                t_test.append((lh_var, rh_var, statistic))
            else:
                print(f"Found {(lh_var, rh_var)} NOT TO BE correlated with T Test={statistic}")
        else:
            statistic, p_value = scpy_stats.f_oneway(*[part[0] for part in numerical_for_each_category])

            if p_value <= alfa:
                print(f"Found {(lh_var, rh_var)} TO BE correlated with ANOVA Test={statistic}")
                anova_test.append((lh_var, rh_var, statistic))
            else:
                print(f"Found {(lh_var, rh_var)} NOT TO BE correlated with ANOVA Test={statistic}")


In [ ]:
t_test = list(sorted(t_test, key=lambda x: x[2]))
print(f"Testul mediilor (T-Test)\n\n")
for item in t_test:
    print(f"{item}\n")

In [ ]:
anova_test = list(sorted(anova_test, key=lambda x: x[2]))
print(f"Testul ANOVA\n\n")
for item in anova_test:
    print(f"{item}\n")

In [ ]:
best_t_test = t_test[:8] + t_test[-4:]

In [ ]:
best_anova_test = anova_test[-14:]

### 4. PCA in 2D si 3D; TSNE in 2d si 3D; Both using numerical + categorial

In [ ]:
numeric_dataset = DatasetManager.obj_list_to_np_array_numeric(dataset_obj_list, NUMERICAL_VARIABLES)
#numeric_dataset = StandardScaler().fit_transform(numeric_dataset)

categorical_to_binary_dataset = DatasetManager.obj_list_to_np_array_category_binary(dataset_obj_list, CATEGORICAL_VARIABLES_NO_LABEL)
numerical_categorical_dataset = np.hstack((numeric_dataset, categorical_to_binary_dataset))
#numerical_categorical_dataset = StandardScaler().fit_transform(numerical_categorical_dataset)

label_dataset = DatasetManager.obj_list_to_np_array(dataset_obj_list, [LABEL_VARIABLE])
label_dataset, label_indexes = pd.factorize(pd.DataFrame(label_dataset)[0])

In [ ]:
numerical_transformed_2d = PCA(n_components=2).fit_transform(numeric_dataset)
numerical_transformed_3d = PCA(n_components=3).fit_transform(numeric_dataset)

numerical_categorial_transformed_2d = PCA(n_components=2).fit_transform(numerical_categorical_dataset)
numerical_categorial_transformed_3d = PCA(n_components=3).fit_transform(numerical_categorical_dataset)

In [ ]:
plt.scatter(numerical_transformed_2d[:, 0], numerical_transformed_2d[:, 1], c=label_dataset, label=label_dataset)
plt.show()
plt.clf()

In [ ]:
plt.scatter(numerical_categorial_transformed_2d[:, 0], numerical_categorial_transformed_2d[:, 1], c=label_dataset)
plt.show()
plt.clf()

### 5. Corrgrams